In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm
import pickle
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: c:\Users\mirun\24_winter\press\sr-press


In [2]:
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names
from functools import partial

from express.datasets import PressingDataset
from tqdm import tqdm
import pandas as pd
from pathlib import Path


In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")
test_db = SQLiteDatabase(TEST_DB_PATH)
dataset_test = partial(PressingDataset, path=os.path.join(base_path, "stores", "filtered_datasets", "test"))


### Team_id를 team_name으로 매핑하기

In [4]:
teams_data = []
seen_team_ids = set()
for idx, game in tqdm(test_db.games().iterrows(), total=len(test_db.games())):
    game_id = idx
    
    # game_id에 해당하는 팀 정보를 가져오기
    teams = SBL.teams(game_id)  
    
    # 각 팀에 대해 고유한 팀 정보를 추출
    for i in range(len(teams)):
        team_id = teams.iloc[i]['team_id']
        team_name = teams.iloc[i]['team_name']
        
        # 중복된 team_id는 추가하지 않음
        if team_id not in seen_team_ids:
            teams_data.append({
                'team_id': team_id,
                'team_name': team_name
            })
            seen_team_ids.add(team_id)  # 추적 집합에 추가하여 중복을 방지

teams_df = pd.DataFrame(teams_data)


100%|██████████| 64/64 [00:34<00:00,  1.86it/s]


30초 정도 걸리지만 teams_2_id.json 파일을 쓰셔도 됩니다.

In [52]:
import json
team_mapping_dict = teams_df.set_index('team_id')['team_name'].to_dict()

# JSON 파일로 저장
with open('teams_2_id.json', 'w') as file:
    json.dump(team_mapping_dict, file)

### Preds값 가져오기

In [24]:
model = "xgboost" #"soccermap"
trial = 2 #as you like

path = Path(base_path) / "stores" / "model" / model / f"{trial:03d}" / "component.pkl"
with path.open(mode="rb") as file:
    component = pickle.load(file)
preds = component.predict(dataset_test)
preds

4       0.035026
14      0.011667
17      0.011756
22      0.015184
34      0.378553
          ...   
3013    0.495724
3020    0.065638
3031    0.455915
3041    0.005586
3045    0.006484
Length: 14504, dtype: float32

In [23]:
test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"),
    xfns=["closest_11_players", "get_column_sum_to_player","dist_opponent","relative_startlocation"],
    yfns=["counterpress"],
    load_cached=True,
    nb_prev_actions=3
)

In [25]:
new=test_dataset.labels.copy()
new['predict_labels'] = preds.values
new


counterpress  predict_labels
game_id action_id                              
3857254 4                 False        0.035026
        14                False        0.011667
        17                False        0.011756
        22                False        0.015184
        34                False        0.378553
...                         ...             ...
3869685 3013               True        0.495724
        3020              False        0.065638
        3031              False        0.455915
        3041              False        0.005586
        3045              False        0.006484

[14504 rows x 2 columns]

### pressing action마다 team_id 찾아서 append하기
- 이것보다 빠르게 구할 수 있는 방법 아신다면 수정해주십시오...
- 50분정도 걸립니다

In [ ]:
from tqdm import tqdm
teams_id = []  # 딕셔너리가 아닌 리스트로 초기화합니다.

# 데이터프레임을 순회하며 각 (game_id, action_id)에 대해 팀 ID를 찾습니다.
for (game_id, action_id), row in tqdm(new.iterrows(), total=len(new)):
   
    # add_names()를 사용하여 특정 game_id에 대한 액션을 가져옵니다.
    actions = add_names(test_db.actions(game_id))

    # 해당 game_id와 action_id를 사용하여 필요한 인덱스의 값을 가져옵니다.
    idx = (game_id, action_id)

    try:
        # 인덱스가 존재할 경우 team_id를 teams_id 리스트에 추가합니다.
        team_id = actions.loc[idx]['team_id']
        teams_id.append(team_id)
    except KeyError:
        # 만약 해당 인덱스가 존재하지 않는다면, None 또는 다른 기본값을 추가합니다.
        teams_id.append(None)


new['team'] = teams_id
new

100%|██████████| 14504/14504 [47:24<00:00,  5.10it/s]


WC_test_team_id.csv로 이미 만들어놨습니다.

In [27]:
loaded_teams_df=pd.read_csv('WC_test_team_id.csv')
new['team_id']=loaded_teams_df['team_id'].values
new

counterpress  predict_labels  team_id
game_id action_id                                       
3857254 4                 False        0.035026      777
        14                False        0.011667      777
        17                False        0.011756      777
        22                False        0.015184      777
        34                False        0.378553      776
...                         ...             ...      ...
3869685 3013               True        0.495724      771
        3020              False        0.065638      771
        3031              False        0.455915      779
        3041              False        0.005586      779
        3045              False        0.006484      779

[14504 rows x 3 columns]

### team_name 매핑해서 append하기

In [28]:
import json
with open('teams_2_id.json', 'r', encoding='utf-8') as file:
    team_mapping = json.load(file)
new['team_name'] = new['team_id'].astype(str).map(team_mapping)
new


counterpress  predict_labels  team_id  team_name
game_id action_id                                                  
3857254 4                 False        0.035026      777    Tunisia
        14                False        0.011667      777    Tunisia
        17                False        0.011756      777    Tunisia
        22                False        0.015184      777    Tunisia
        34                False        0.378553      776    Denmark
...                         ...             ...      ...        ...
3869685 3013               True        0.495724      771     France
        3020              False        0.065638      771     France
        3031              False        0.455915      779  Argentina
        3041              False        0.005586      779  Argentina
        3045              False        0.006484      779  Argentina

[14504 rows x 4 columns]

In [29]:
# 팀당 pressing point 합 구하기
team_sum = new.groupby('team_name')['predict_labels'].sum().reset_index()

# 팀당 서로 다른 game_id 개수 세기
team_game_count = new.reset_index().groupby('team_name')['game_id'].nunique()

team_game_count_df = team_game_count.reset_index()

# 두 데이터프레임 병합 (team_name을 기준으로 병합)
merged_df = pd.merge(team_sum, team_game_count_df, on='team_name')


merged_df.rename(columns={'game_id': 'games_played'}, inplace=True)
merged_df['points_per_match'] = (merged_df['predict_labels'] / merged_df['games_played']).round(2)
merged_df.sort_values('points_per_match', ascending=False, inplace=True)
merged_df

,team_name,predict_labels,games_played,points_per_match
12,Germany,81.591766,3,27.20
26,Spain,100.260521,4,25.07
4,Cameroon,73.595573,3,24.53
15,Japan,95.922440,4,23.98
17,Morocco,164.282959,7,23.47
29,United States,92.529160,4,23.13
5,Canada,67.091316,3,22.36
0,Argentina,156.116898,7,22.30
7,Croatia,155.779449,7,22.25
16,Mexico,66.719841,3,22.24
